# Import Libraries

In [12]:
import os
import copy
from tqdm.notebook import tqdm
import pprint
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import torch
from torch import nn
import os
import math
from src.models.utils import cal_l1_loss
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.core.lightning import LightningModule
from easydict import EasyDict as edict
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from src.constants import DATA_PATH, MASTER_THESIS_DIR, TRAINING_CONFIG_PATH, SIMCLR_CONFIG, SSL_CONFIG, PAIRWISE_CONFIG, MODEL_CONFIG_PATH
from src.data_loader.data_set import Data_Set
from src.data_loader.utils import get_train_val_split
from src.experiments.utils import get_experiement_args, process_experiment_args
from src.models.callbacks.upload_comet_logs import UploadCometLogs
from src.models.simclr_model import SimCLR
from src.utils import get_console_logger, read_json
from torchvision import transforms
from pl_bolts.optimizers.lars_scheduling import LARSWrapper
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from src.models.supervised_head_model import SupervisedHead

# Read data

In [13]:
train_param = edict(read_json(TRAINING_CONFIG_PATH))
model_param = edict(
        read_json(SSL_CONFIG
        )
)
train_param.augmentation_flags.resize=True
train_param.augmentation_flags.rotate=False
train_param.augmentation_flags.crop=True

In [14]:
train_param.num_workers=8
train_param.epochs = 150
model_param.warmup_epochs = 10
train_param, model_param

({'batch_size': 128,
  'epochs': 150,
  'train_ratio': 0.9,
  'gpu': True,
  'num_workers': 8,
  'seed': 5,
  'augmentation_flags': {'color_drop': True,
   'color_jitter': True,
   'crop': True,
   'cut_out': True,
   'flip': False,
   'gaussian_blur': True,
   'random_crop': False,
   'resize': True,
   'rotate': False},
  'augmentation_order': [],
  'augmentation_flags0': {'color_drop': True,
   'color_jitter': True,
   'crop': True,
   'cut_out': True,
   'flip': False,
   'gaussian_blur': True,
   'random_crop': False,
   'resize': True,
   'rotate': True},
  'augmentation_params': {'crop_margin': 1.5,
   'crop_margin_range': [0.6, 2.0],
   'cut_out_fraction': [0.0, 0.16],
   'hue_factor_range': [0.01, 1.0],
   'max_angle': 359,
   'min_angle': 0,
   'resize_shape': [128, 128],
   'sat_factor_range': [0.01, 1.0],
   'value_factor_alpha_range': [0.5, 1],
   'value_factor_beta_range': [5, 20],
   'crop_box_jitter': [0.0, 0.1]}},
 {'batch_size': 128,
  'lr': 0.0001,
  'opt_weight_deca

In [15]:
data = Data_Set(
        config=train_param,
        transform=transforms.Compose([transforms.ToTensor()]),
        train_set=True,
        experiment_type="supervised",
    )
model_param.num_samples= len(data)
model_param.alpha =5
model_param.gpu = True

In [16]:
model_param.saved_model_name = "9f302cea72a2459eb14dfad33b1a9783"
model_param.checkpoint=""

In [17]:
train_data_loader, val_data_loader = get_train_val_split(
        data, batch_size=train_param.batch_size, num_workers=train_param.num_workers
    )

In [7]:
model_param

{'batch_size': 128,
 'lr': 0.0001,
 'opt_weight_decay': 0.0001,
 'warmup_epochs': 10,
 'simclr_experiment_name': '14910753afb9499ab9229174d6377efb',
 'alpha': 5,
 'gpu': True,
 'checkpoint': '',
 'num_samples': 117216,
 'saved_model_name': '9f302cea72a2459eb14dfad33b1a9783'}

In [18]:
from src.models.pairwise_model import SupervisedHead
model_ssl = SupervisedHead(model_param)

In [9]:
model_ssl.config.warmup_epochs

10

In [21]:
comet_logger = CometLogger(
    api_key=os.environ.get("COMET_API_KEY"),
    project_name="master-thesis",
    workspace="dahiyaaneesh",
    save_dir=os.path.join(DATA_PATH, "models"),
)
upload_comet_logs = UploadCometLogs(
    "step", get_console_logger("callback"), "supervised_ssl"
)
lr_monitor = LearningRateMonitor(logging_interval="step")
trainer = Trainer(precision=16,
    gpus=1,
    max_epochs=train_param.epochs,
#     logger=comet_logger,
    callbacks=[upload_comet_logs,lr_monitor],
)

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [22]:
trainer.fit(model_ssl,train_data_loader, val_data_loader)

callback -ERROR - Comet logger object missing! Logs won't be updated
callback -ERROR - Comet logger object missing! Logs won't be updated
callback -ERROR - Comet logger object missing! Logs won't be updated

  | Name         | Type       | Params
--------------------------------------------
0 | encoder      | ResNet     | 11 M  
1 | final_layers | Sequential | 74 K  


/local/home/adahiya/Documents/master_thesis/master_thesis_env/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b